In [ ]:
# %load ./init.ipy
%reload_ext autoreload
%autoreload 2
from importlib import reload

#Need py38 environment for this to run

import sys
import os
import sys
import logging
import warnings
import numpy as np
import astropy as ap
import scipy as sp
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt

import h5py
import tqdm.notebook as tqdm

import kalepy as kale
import kalepy.utils
import kalepy.plot

import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
plt.rcParams.update({'grid.alpha': 0.5})

log = holo.log
log.setLevel(logging.INFO)

# ---- Create initial population

pm_ecc = holo.population.PM_Eccentricity([0.2,0.2])
pop = holo.population.Pop_Illustris(mods=pm_ecc)

# ---- Apply population modifiers
redz = cosmo.a_to_z(pop.scafa)

hards_no_eccen = [
    holo.evolution.Hard_GW,
    holo.evolution.Sesana_Scattering(),
    holo.evolution.Dynamical_Friction_NFW(attenuate=False),
]

hards_eccen = [
    holo.evolution.Hard_GW,
    holo.evolution.CBD_Torques(),
    holo.evolution.Sesana_Scattering(),
    holo.evolution.Dynamical_Friction_NFW(attenuate=False),
]

f_edd = 0.10
eccen = 0

acc_instance = holo.accretion.Accretion(accmod='Siwek22', f_edd = f_edd, eccen=eccen, subpc=True)

keys = ['no_doteb', 'doteb']
evol_dict = {}
for key,hards in zip(keys,[hards_no_eccen, hards_eccen]):
    print("hards = ", hards)
    evo = holo.evolution.Evolution(pop, hards, nsteps = 100, debug=True, acc=acc_instance)
    evo.evolve()
    evol_dict[key] = evo



In [ ]:
labels = [r'$\dot{e}_{\rm b,CBD} = 0$', r'$\dot{e}_{\rm b,CBD} \, \rm{Siwek+23}$']

#Take a few random numbers and plot their mass evolutions as a function of time
import random
fs = 30

n_mbhb = evo.shape[0]

n_mbhb_to_plot = 2
inds_random_mbhb = []

p = 0
while p < n_mbhb_to_plot:
    x = random.randint(0,n_mbhb)
    inds_t_pos = evo.tlook[x] > 0
    if len(evo.tlook[x][inds_t_pos]) > 10:
        inds_random_mbhb.append(x)
        p += 1


fig, axs = plt.subplots(int(n_mbhb_to_plot),2,figsize=(25,15))
fig.suptitle('Eccentricities and Mass Ratios of MBHBs', fontsize=fs)
k=0
for i in range(0,int(n_mbhb_to_plot)):
    for key,label in zip(keys,labels):
        evo = evol_dict[key]
        inds_t_pos = evo.tlook[inds_random_mbhb[k]] > 0
        time = (evo.tlook[inds_random_mbhb[k]][inds_t_pos])
        masses = (evo.mass[inds_random_mbhb[k]][inds_t_pos])
        separation = (evo.sepa[inds_random_mbhb[k]][inds_t_pos])
        print("evo.eccen = ", evo.eccen)
        dadt = -(evo.dadt[inds_random_mbhb[k]][inds_t_pos])
        eccen = evo.eccen[inds_random_mbhb[k]][inds_t_pos]
        dedt = evo.dedt[inds_random_mbhb[k]][inds_t_pos]
        #PLOT SEPARATIONS
        j = 0
        axs[i][j].loglog(separation/PC, eccen, '*-', label=label)
        axs[i][j].set_xlabel("separation [PC]", fontsize=fs)
        axs[i][j].set_ylabel(r'$e_{\rm{b}}$', fontsize=fs)
        #PLOT MASS RATIO
        j = 1
        if masses.T[0][0] >= masses.T[1][0]:
            m1 = masses.T[0]
            m2 = masses.T[1]
        else:
            m1 = masses.T[1]
            m2 = masses.T[0]
        q_b = np.array(m2)/np.array(m1)
        axs[i][j].loglog(separation/PC, q_b, '*-', label=label)
        axs[i][j].set_xlabel(r'$a_{\rm{b}} \, [\rm{PC}]$', fontsize=fs)
        axs[i][j].set_ylabel(r'$q_{\rm{b}}$', fontsize=fs)
#         j = 2
#         axs[i][j].loglog(separation/PC, -dedt, '*-', label=label)
#         axs[i][j].set_xlabel("separation [PC]", fontsize=fs)
#         axs[i][j].set_ylabel(r'$\dot{e}_{\rm{b}}$', fontsize=fs)
        
        for j in [0,1]:
            #axs[i][j].invert_xaxis()
            axs[i][j].legend(fontsize=fs)
            axs[i][j].set_xlim([1.e-4, 1.e2])
            plt.setp(axs[i][j].get_xticklabels(which='both'), fontsize=fs, rotation=45)
            plt.setp(axs[i][j].get_yticklabels(which='both'), fontsize=fs)
            ticksize = 15
            tickwidth = 2
            axs[i][j].tick_params(axis='both', which='major', direction='inout', size=ticksize, width=tickwidth)
            axs[i][j].tick_params(axis='both', which='minor', direction='inout', size=0.5*ticksize, width=0.25*tickwidth)

    k+=1
    plt.tight_layout()
    

# NOW PLOT THE MASS RATIO VS ECCENTRICITY FOR BINARIES:
## - initial/final (irrespective whether merger has occurred)
## - AT Z = 0, NOT MERGED (LOOK FOR -ve REDSHIFTS?)

# initial/final eccentricity AT MERGER

In [ ]:
data_dict = {}
fields = ['q_init', 'q_final', 'e_init', 'e_final']
for f in fields:
    for key in keys:
        data_dict['%s_%s' %(f,key)] = []

for key in keys:
    evo = evol_dict[key]
    masses = evo.mass
    for i in np.arange(0,np.shape(masses)[0]):
        inds_t_pos = evo.tlook[i] > 0
        if len(evo.tlook[i][inds_t_pos]) > 5:
            masses = evo.mass[i][inds_t_pos]
            if masses.T[0][0] > masses.T[1][0]:
                primary_init = masses.T[0][0]
                secondary_init = masses.T[1][0]
            else:
                primary_init = masses.T[1][0]
                secondary_init = masses.T[0][0]
            if masses.T[0][-1] > masses.T[1][-1]:
                primary_final = masses.T[0][-1]
                secondary_final = masses.T[1][-1]
            else:
                primary_final = masses.T[1][-1]
                secondary_final = masses.T[0][-1]
            q_init = secondary_init/primary_init
            q_final = secondary_final/primary_final
            eccen_initial = evo.eccen[i][inds_t_pos][0]
            eccen_final = evo.eccen[i][inds_t_pos][-1]
            
            if q_init>=0.1:
                data_dict['q_init_%s' %key].append(q_init)
                data_dict['q_final_%s' %key].append(q_final)
                data_dict['e_init_%s' %key].append(eccen_initial)
                data_dict['e_final_%s' %key].append(eccen_final)
                

In [ ]:
import random
import seaborn as sns
palette = sns.color_palette('bright')
all_colors = {}
for i,key in enumerate(keys):
    all_colors[key] = palette[i]

fs = 25
nbins = 100
bins = np.linspace(1.e-1, 1.0, nbins)
logbins = bins

fig, axs = plt.subplots(len(keys)+1,1,figsize=(15,20))
for ax,key,label in zip(axs,keys,labels):
    ax.hist(data_dict['e_init_%s' %key], bins = logbins, log=True, \
            label=r'$e_{\rm b,init},$' + ' mean=%.2f' %(np.mean(data_dict['e_init_%s' %key])), \
            alpha=0.7, color='gray')
    ax.hist(data_dict['e_final_%s' %key], bins = logbins, log=True, \
            label=r'$e_{\rm b,final},$' + ' mean=%.2f' %(np.mean(data_dict['e_final_%s' %key])), \
            alpha=0.7, color=all_colors[key])
    ax.legend(fontsize=fs)
    ax.set_title(label, fontsize=fs)
    plt.setp(ax.get_xticklabels(which='both'), fontsize=fs, rotation=45)
    plt.setp(ax.get_yticklabels(which='both'), fontsize=fs)

for key,label in zip(keys,labels):
    ax = axs[-1]
    ax.hist(data_dict['e_final_%s' %key], bins = logbins, log=True, \
            label=label + ', ' + r'$e_{\rm b,final},$' + ' mean=%.2f' %(np.mean(data_dict['e_final_%s' %key])), \
            alpha=0.7, color=all_colors[key])
    ax.legend(fontsize=fs)
    ax.set_title("Final Eccentricity Distributions in Holodeck given Siwek+23 evolution models", fontsize=fs)
    plt.setp(ax.get_xticklabels(which='both'), fontsize=fs, rotation=45)
    plt.setp(ax.get_yticklabels(which='both'), fontsize=fs)
    ax.set_xlabel("Binary Eccentricity", fontsize=fs)
    plt.tight_layout()

# fname = "mass_ratio_evolution_subpc_f_edd%.2f_eb" %(f_edd)
# for eccen in all_eccen:
#     fname += "_%.2f" %eccen
# plt.savefig(fname + ".png", facecolor='w')
# import os
# print("os.getcwd() = ", os.getcwd())

# MASS RATIO DISTRIBUTION

In [ ]:
import random
import seaborn as sns
palette = sns.color_palette('bright')
all_colors = {}
for i,key in enumerate(keys):
    all_colors[key] = palette[i]

fs = 25
nbins = 100
bins = np.linspace(1.e-1, 1.0, nbins)
logbins = bins

fig, axs = plt.subplots(len(keys)+1,1,figsize=(15,20))
for ax,key,label in zip(axs,keys,labels):
    ax.hist(data_dict['q_init_%s' %key], bins = logbins, log=True, \
            label=r'$q_{\rm b,init},$' + ' mean=%.2f' %(np.mean(data_dict['q_init_%s' %key])), \
            alpha=0.7, color='gray')
    ax.hist(data_dict['q_final_%s' %key], bins = logbins, log=True, \
            label=r'$q_{\rm b,final},$' + ' mean=%.2f' %(np.mean(data_dict['q_final_%s' %key])), \
            alpha=0.7, color=all_colors[key])
    ax.legend(fontsize=fs)
    ax.set_title(label, fontsize=fs)
    plt.setp(ax.get_xticklabels(which='both'), fontsize=fs, rotation=45)
    plt.setp(ax.get_yticklabels(which='both'), fontsize=fs)

for key,label in zip(keys,labels):
    ax = axs[-1]
    ax.hist(data_dict['q_final_%s' %key], bins = logbins, log=True, \
            label=label + ', ' + r'$q_{\rm b,final},$' + ' mean=%.2f' %(np.mean(data_dict['q_final_%s' %key])), \
            alpha=0.7, color=all_colors[key])
    ax.legend(fontsize=fs)
    ax.set_title("Final Mass Ratio Distributions in Holodeck given Siwek+22 accretion models", fontsize=fs)
    plt.setp(ax.get_xticklabels(which='both'), fontsize=fs, rotation=45)
    plt.setp(ax.get_yticklabels(which='both'), fontsize=fs)
    ax.set_xlabel("Binary Mass Ratio", fontsize=fs)
    plt.tight_layout()

# fname = "mass_ratio_evolution_subpc_f_edd%.2f_eb" %(f_edd)
# for eccen in all_eccen:
#     fname += "_%.2f" %eccen
# plt.savefig(fname + ".png", facecolor='w')
# import os
# print("os.getcwd() = ", os.getcwd())

# AT Z = 0, NOT MERGED

In [ ]:
data_dict = {}
fields = ['q_init', 'q_final', 'e_init', 'e_final']
for f in fields:
    for key in keys:
        data_dict['%s_%s' %(f,key)] = []

for key in keys:
    evo = evol_dict[key]
    masses = evo.mass
    separations = evo.sepa
    for i in np.arange(0,np.shape(masses)[0]):
        inds_t_pos = evo.tlook[i] > 0
        if len(evo.tlook[i][inds_t_pos]) > 5:
            masses = evo.mass[i][inds_t_pos]
            separations = evo.sepa[i][inds_t_pos]
            """ Find where binary is at 0.001 parsec, record eccentricity then """
            def closest_idx(lst, K):
                return(np.where(lst == lst[min(range(len(lst)), key = lambda i: abs(lst[i]-K))]))

            idx = closest_idx(separations, 0.005*PC)[0][0]
            print("idx = ", idx)
            
            if masses.T[0][0] > masses.T[1][0]:
                primary_init = masses.T[0][0]
                secondary_init = masses.T[1][0]
            else:
                primary_init = masses.T[1][0]
                secondary_init = masses.T[0][0]
            if masses.T[0][-1] > masses.T[1][-1]:
                primary_final = masses.T[0][idx]
                secondary_final = masses.T[1][idx]
            else:
                primary_final = masses.T[1][idx]
                secondary_final = masses.T[0][idx]
            
            q_init = secondary_init/primary_init
            q_final = secondary_final/primary_final
            eccen_initial = evo.eccen[i][inds_t_pos][0]
            eccen_final = evo.eccen[i][inds_t_pos][idx]
            print("q_final = ", q_final)
            print("eccen_final = ", eccen_final)
                        
            data_dict['q_init_%s' %key].append(q_init)
            data_dict['q_final_%s' %key].append(q_final)
            data_dict['e_init_%s' %key].append(eccen_initial)
            data_dict['e_final_%s' %key].append(eccen_final)
                

In [ ]:
import random
import seaborn as sns
palette = sns.color_palette('bright')
all_colors = {}
for i,key in enumerate(keys):
    all_colors[key] = palette[i]

fs = 25
nbins = 100
bins = np.linspace(1.e-1, 1.0, nbins)
logbins = bins

fig, axs = plt.subplots(len(keys)+1,1,figsize=(15,20))
for ax,key,label in zip(axs,keys,labels):
    ax.hist(data_dict['e_init_%s' %key], bins = logbins, log=True, \
            label=r'$e_{\rm b,init},$' + ' mean=%.2f' %(np.mean(data_dict['e_init_%s' %key])), \
            alpha=0.7, color='gray')
    ax.hist(data_dict['e_final_%s' %key], bins = logbins, log=True, \
            label=r'$e_{\rm b,final},$' + ' mean=%.2f' %(np.mean(data_dict['e_final_%s' %key])), \
            alpha=0.7, color=all_colors[key])
    ax.legend(fontsize=fs)
    ax.set_title(label, fontsize=fs)
    plt.setp(ax.get_xticklabels(which='both'), fontsize=fs, rotation=45)
    plt.setp(ax.get_yticklabels(which='both'), fontsize=fs)

for key in keys:
    ax = axs[-1]
    ax.hist(data_dict['e_final_%s' %key], bins = logbins, log=True, \
            label=label + ', ' + r'$e_{\rm b,final},$' + ' mean=%.2f' %(np.mean(data_dict['e_final_%s' %key])), \
            alpha=0.7, color=all_colors[key])
    ax.legend(fontsize=fs)
    ax.set_title("Eccentricity Distributions at 0.005pc given Siwek+23b accretion models", fontsize=fs)
    plt.setp(ax.get_xticklabels(which='both'), fontsize=fs, rotation=45)
    plt.setp(ax.get_yticklabels(which='both'), fontsize=fs)
    ax.set_xlabel("Binary Eccentricity", fontsize=fs)
    plt.tight_layout()

fname = "eccentricity_evolution_0.005pc"
plt.savefig(fname + ".png", facecolor='w')
import os
print("os.getcwd() = ", os.getcwd())

In [ ]:
keys = ['no_doteb', 'doteb']
import random
import seaborn as sns
palette = sns.color_palette('bright')
all_colors = {}
for i,key in enumerate(keys):
    all_colors[key] = palette[i]

fs = 30
nbins = 100
bins = np.linspace(1.e-1, 1.0, nbins)
logbins = bins

all_keys = [[keys[0]], keys]
print("all_keys = ", all_keys)

for keys in all_keys:
    fig, ax = plt.subplots(1,1,figsize=(17,10))
    fname = "eccentricity_evolution_0.005pc"
    for label,key in zip(labels,keys):
        fname += '_%s' %key
        ax.hist(data_dict['e_final_%s' %key], bins = logbins, log=True, \
                label=label + ', ' + r'$e_{\rm b,final},$' + ' mean=%.2f' %(np.mean(data_dict['e_final_%s' %key])), \
                alpha=0.7, color=all_colors[key])
        ax.legend(fontsize=fs)
        ax.set_title("Illustris MBHBs: Eccentricity Distributions at 0.005pc", fontsize=fs)
        plt.setp(ax.get_xticklabels(which='both'), fontsize=fs, rotation=45)
        plt.setp(ax.get_yticklabels(which='both'), fontsize=fs)
        ax.set_xlabel("Binary Eccentricity", fontsize=fs)
        plt.tight_layout()

    plt.savefig(fname + ".png", facecolor='w')
    import os
    print("os.getcwd() = ", os.getcwd())

# what is the highest mass system detectable by LISA in this sample just before coalescence?
# compare with chirp masses in Zrake+21 and see if harmonics might change observability in high mass binaries

# PLOT BINARY ECCENTRICITY AGAINST MASS RATIO

In [ ]:
import random
import seaborn as sns
palette = sns.color_palette('bright')
all_colors = {}
for i,key in enumerate(keys):
    all_colors[key] = palette[i]

fs = 25
nbins = 100
bins = np.linspace(1.e-1, 1.0, nbins)
logbins = bins

fig, axs = plt.subplots(len(keys)+1,1,figsize=(15,20))
for ax,key,label in zip(axs,keys,labels):
    ax.plot(data_dict['q_init_%s' %key], data_dict['e_init_%s' %key], '.', \
            label=label + ', init', \
            alpha=0.7, color='grey')
    ax.plot(data_dict['q_final_%s' %key], data_dict['e_final_%s' %key], '.', \
            label=label + ', final', \
            alpha=0.7, color=all_colors[key])
    ax.legend(fontsize=fs)
    ax.set_title(label, fontsize=fs)
    plt.setp(ax.get_xticklabels(which='both'), fontsize=fs, rotation=45)
    plt.setp(ax.get_yticklabels(which='both'), fontsize=fs)
    ax.set_xlim([1.e-1,1.0])

for key,label in zip(keys,labels):
    ax = axs[-1]
    ax.plot(data_dict['q_final_%s' %key], data_dict['e_final_%s' %key], '.', \
            label=label, \
            alpha=0.7, color=all_colors[key])
    ax.legend(fontsize=fs)
    ax.set_title("Final Mass Ratio Distributions in Holodeck given Siwek+22 accretion models", fontsize=fs)
    plt.setp(ax.get_xticklabels(which='both'), fontsize=fs, rotation=45)
    plt.setp(ax.get_yticklabels(which='both'), fontsize=fs)
    ax.set_xlabel("Binary Eccentricity", fontsize=fs)
    ax.set_xlim([1.e-1,1.0])
    plt.tight_layout()

# fname = "mass_ratio_evolution_subpc_f_edd%.2f_eb" %(f_edd)
# for eccen in all_eccen:
#     fname += "_%.2f" %eccen
# plt.savefig(fname + ".png", facecolor='w')
# import os
# print("os.getcwd() = ", os.getcwd())

In [ ]:
#Take a few random numbers and plot their mass evolutions as a function of time
import random
fs = 30

n_mbhb = evo.shape[0]

n_mbhb_to_plot = 2
inds_random_mbhb = []

p = 0
while p < n_mbhb_to_plot:
    x = random.randint(0,n_mbhb)
#     inds_random_mbhb.append(x)
#     p+=1
    inds_t_pos = evo.tlook[x] > 0
    if len(evo.tlook[x][inds_t_pos]) > 5:
        inds_random_mbhb.append(x)
        p += 1


fig, axs = plt.subplots(int(n_mbhb_to_plot),3,figsize=(25,15))
fig.suptitle('Eccentricities and Mass Ratios of MBHBs', fontsize=fs)
k = 0
for i in range(0,int(n_mbhb_to_plot)):
    #for j in range(0,int(n_mbhb_to_plot/2)):
    inds_t_pos = evo.tlook[inds_random_mbhb[k]] > 0
    time = (evo.tlook[inds_random_mbhb[k]][inds_t_pos])
    masses = (evo.mass[inds_random_mbhb[k]][inds_t_pos])
    separation = (evo.sepa[inds_random_mbhb[k]][inds_t_pos])
    dadt = -(evo.dadt[inds_random_mbhb[k]][inds_t_pos])
    eccen = evo.eccen[inds_random_mbhb[k]][inds_t_pos]
    dedt = evo.dedt[inds_random_mbhb[k]][inds_t_pos]
    #PLOT SEPARATIONS
    j = 0
    axs[i][j].loglog(separation/PC, eccen, '*-')
    axs[i][j].set_xlabel("separation [PC]", fontsize=fs)
    #axs[i][j].set_ylabel(r'$a_{\rm{b}} \, [\rm{PC}]$', fontsize=fs)
    axs[i][j].set_ylabel(r'$e_{\rm{b}}$', fontsize=fs)
    #axs[i][j].invert_xaxis()
    #axs[i][j].legend(fontsize=fs)
    plt.setp(axs[i][j].get_xticklabels(which='both'), fontsize=fs, rotation=45)
    plt.setp(axs[i][j].get_yticklabels(which='both'), fontsize=fs)
    #PLOT MASSES
    j = 1
    if masses.T[0][0] >= masses.T[1][0]:
        m1 = masses.T[0]
        m2 = masses.T[1]
    else:
        m1 = masses.T[1]
        m2 = masses.T[0]
    q_b = np.array(m2)/np.array(m1)
    axs[i][j].loglog(separation/PC, q_b, '*-')
    #axs[i][j].set_xlabel("lookback time [GYR]", fontsize=fs) 
    axs[i][j].set_xlabel(r'$a_{\rm{b}} \, [\rm{PC}]$', fontsize=fs)
    axs[i][j].set_ylabel(r'$q_{\rm{b}}$', fontsize=fs)
    #axs[i][j].invert_xaxis()
    #axs[i][j].legend(fontsize=fs)
    plt.setp(axs[i][j].get_xticklabels(which='both'), fontsize=fs, rotation=45)
    plt.setp(axs[i][j].get_yticklabels(which='both'), fontsize=fs)
    j = 2
    axs[i][j].loglog(separation/PC, -dedt, '*-')
    axs[i][j].set_xlabel("separation [PC]", fontsize=fs)
    #axs[i][j].set_ylabel(r'$a_{\rm{b}} \, [\rm{PC}]$', fontsize=fs)
    axs[i][j].set_ylabel(r'$\dot{e}_{\rm{b}}$', fontsize=fs)
    #axs[i][j].invert_xaxis()
    #axs[i][j].legend(fontsize=fs)
    plt.setp(axs[i][j].get_xticklabels(which='both'), fontsize=fs, rotation=45)
    plt.setp(axs[i][j].get_yticklabels(which='both'), fontsize=fs)
    k+=1

        
plt.tight_layout()
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
